In [2]:
import torch
import numpy as np
import cvxopt as opt
from cvxopt import solvers
import os

from models.MVO import MVO
from data.NewETFs import NewETFs
from utils.dataset_utils import create_rolling_window_ts, timeseries_train_test_split

In [3]:
model_name = "mvo"
train_ratio = 0.6
num_timesteps_in = 252
num_timesteps_out = 1
fix_start = False
drop_last = True

# relevant paths
# source_path = os.path.dirname(__file__)
source_path = os.getcwd()
inputs_path = os.path.join(source_path, "data", "inputs")

# prepare dataset
loader = NewETFs(use_last_data=True, use_first_50_etfs=True)
prices = loader.prices.T
returns = loader.returns.T
features = loader.features
features = features.reshape(features.shape[0], features.shape[1] * features.shape[2]).T  

X_steps, prices_steps = create_rolling_window_ts(features=features, 
                                                    target=prices,
                                                    num_timesteps_in=num_timesteps_in,
                                                    num_timesteps_out=num_timesteps_out,
                                                    fix_start=fix_start,
                                                    drop_last=drop_last)

# call model
model = MVO()

# training/validation + oos testing
test_weights = []
train_loss = val_loss = test_loss = []
# for step in range(X_steps.shape[0]):
#     X_t = X_steps[step, :, :]
#     prices_t1 = prices_steps[step, :, :]

#     weights = model.forward(X_t)

step = 0
X_t = X_steps[step, :, :]
prices_t1 = prices_steps[step, :, :]

In [5]:
X_t

tensor([[-0.0317,  0.0032, -0.0060,  ...,  0.0347, -0.0237,  0.0533],
        [ 0.0508,  0.0075,  0.0087,  ..., -0.0161, -0.0272, -0.0090],
        [-0.0214,  0.0000, -0.0002,  ..., -0.0089,  0.0053, -0.0052],
        ...,
        [ 0.0118,  0.0059, -0.0086,  ..., -0.0025,  0.0061, -0.0140],
        [ 0.0263,  0.0156, -0.0099,  ...,  0.0310,  0.0093,  0.0208],
        [ 0.0446, -0.0185, -0.0511,  ..., -0.0141, -0.0389, -0.0082]])

In [33]:
risk_aversion = 1
T = X_t.shape[0]
n = X_t.shape[1]

# mle of the mean
mean_t = torch.mean(X_t, axis=0)

# mle of the covariance
cov_t = torch.matmul((X_t - mean_t).T, (X_t - mean_t)) / T

# constraint 1: w_i >= 0 <=> -w_i <= 0, for all i
c1 = torch.eye(n) * -1
h = torch.zeros((n, 1))

# constraint 2: \sum w_i = 1
c2 = torch.ones((1, n))
b = 1.0

# convert to cvxopt matrices
P = opt.matrix(cov_t.numpy().astype(np.double))
q = opt.matrix(mean_t.numpy().astype(np.double))
G = opt.matrix(c1.numpy().astype(np.double))
h = opt.matrix(h.numpy().astype(np.double)) 
A = opt.matrix(c2.numpy().astype(np.double))
b = opt.matrix(b)

In [34]:
opt_output = solvers.qp(P=(risk_aversion * P), q=(-1 * q), G=G, h=h, A=A, b=b) # minimizes the objective
wt = torch.tensor(np.array(opt_output["x"]))

     pcost       dcost       gap    pres   dres
 0: -6.6226e-04 -1.0040e+00  1e+00  0e+00  7e+00
 1: -6.6475e-04 -1.4026e-02  1e-02  2e-17  1e-01
 2: -8.2429e-04 -4.1095e-03  3e-03  1e-16  2e-02
 3: -2.3461e-03 -5.7395e-03  3e-03  4e-16  2e-02
 4: -3.1115e-03 -3.5403e-03  4e-04  2e-16  2e-03
 5: -3.4973e-03 -3.5044e-03  7e-06  2e-16  4e-07
 6: -3.5038e-03 -3.5039e-03  7e-08  1e-16  4e-09
Optimal solution found.


In [35]:
torch.sum(wt)

tensor(1.0000, dtype=torch.float64)